### Graph Theory Assignment 4
> Kaushal Lodd - BT19CSE052

> Saarang Rajguru - BT19CSE092

*Importing Libraries*

In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import statistics
import copy
import time
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import pdist, squareform, cdist
import networkx as nx
from networkx.algorithms import tree

*Reading Dataset*

In [62]:
X = pd.read_csv('../cleaned_dataset/merged_dataset.csv')
Y = pd.read_csv('../dataset/actual.csv')

print(X.shape)
# Remove first column
nodes = np.delete(X.values, 0, 1)
print(nodes)
X.head()


(72, 7130)
[[-214 -153  -58 ...   36  191  -37]
 [-139  -73   -1 ...   11   76  -14]
 [ -76  -49 -307 ...   41  228  -41]
 ...
 [ -55  -44   12 ...   -4  176   40]
 [ -59 -114   23 ...    0   74  -12]
 [-131 -126  -50 ...   34  237   -2]]


,Patient,AFFX-BioB-5_at,AFFX-BioB-M_at,AFFX-BioB-3_at,AFFX-BioC-5_at,AFFX-BioC-3_at,AFFX-BioDn-5_at,AFFX-BioDn-3_at,AFFX-CreX-5_at,AFFX-CreX-3_at,...,U48730_at,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at
0,1,-214,-153,-58,88,-295,-558,199,-176,252,...,185,511,-125,389,-37,793,329,36,191,-37
1,2,-139,-73,-1,283,-264,-400,-330,-168,101,...,169,837,-36,442,-17,782,295,11,76,-14
2,3,-76,-49,-307,309,-376,-650,33,-367,206,...,315,1199,33,168,52,1138,777,41,228,-41
3,4,-135,-114,265,12,-419,-585,158,-253,49,...,240,835,218,174,-110,627,170,-50,126,-91
4,5,-106,-125,-76,168,-230,-284,4,-122,70,...,156,649,57,504,-26,250,314,14,56,-25


In [63]:
print(Y['cancer'].value_counts())
Y.head()

ALL    47
AML    25
Name: cancer, dtype: int64


,patient,cancer
0,1,ALL
1,2,ALL
2,3,ALL
3,4,ALL
4,5,ALL


*Complete Graph*

In [64]:
G = nx.Graph()
dist_matrix = squareform(pdist(nodes))

# Create a Complete Graph
for i in range(len(nodes)):
    for j in range(i+1, len(nodes)):
        G.add_edge(i, j, weight=dist_matrix[i][j])

# Print adjacency matrix
np.set_printoptions(precision=3)
print(dist_matrix.shape)
print(dist_matrix)

(72, 72)
[[     0.     84238.563  88486.177 ...  83298.155  70427.452  60936.541]
 [ 84238.563      0.     84387.974 ... 102642.753  82742.1    83679.759]
 [ 88486.177  84387.974      0.    ... 102304.189 103878.755  94722.794]
 ...
 [ 83298.155 102642.753 102304.189 ...      0.     82128.374  72886.682]
 [ 70427.452  82742.1   103878.755 ...  82128.374      0.     53675.325]
 [ 60936.541  83679.759  94722.794 ...  72886.682  53675.325      0.   ]]


*Constructing 3-neighbourhood MST*

In [65]:
def MST(adj):
    n=len(adj)
    Mst=[]
    start=0
    pq=[]
    mset=set()
    mset.add(start)
    for u in range(n):
        t=[]
        t.append(adj[start][u])
        t.append(start)
        pq.append(t)
    for i in range(n-1):
        node=-1
        val=math.inf
        for u in range(n):
            if u not in mset and pq[u][0]<val:
                val=pq[u][0]
                node=u
        Mst.append([pq[node][1],node])
        mset.add(node)
        for u in range(n):
            if u not in mset and adj[node][u]<pq[u][0]:
                pq[u][0]=adj[node][u]
                pq[u][1]=node
    return Mst

def mstNeighbourhood(graph,t):
    edges=set()
    g=copy.deepcopy(graph)
    for i in range(t):
        result=MST(g)
        for edge in result:
            g[edge[0]][edge[1]]=g[edge[1]][edge[0]]=math.inf
            edges.add(tuple(edge))
    return edges

mst3=mstNeighbourhood(dist_matrix,3)
print(len(mst3))

213


*Adjacency Matrix*

In [66]:
# Comput adjacency matrix for mst3
adj_matrix = np.zeros((len(nodes), len(nodes)))
for edge in mst3:
    adj_matrix[edge[0]][edge[1]] = 1
    adj_matrix[edge[1]][edge[0]] = 1

# Print adjacency matrix
print(adj_matrix.shape)
print(adj_matrix)

(72, 72)
[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 1. 0.]]


*Degree Matrix*

In [67]:
# Compute degree matrxi of mst3
degree_matrix = np.zeros((len(nodes), len(nodes)))
for i in range(len(nodes)):
    degree_matrix[i][i] = sum(adj_matrix[i])

# Print degree matrix
print(degree_matrix.shape)
print(degree_matrix)

(72, 72)
[[ 8.  0.  0. ...  0.  0.  0.]
 [ 0.  4.  0. ...  0.  0.  0.]
 [ 0.  0.  5. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 12.  0.  0.]
 [ 0.  0.  0. ...  0.  7.  0.]
 [ 0.  0.  0. ...  0.  0. 12.]]


*Laplacian Matrix*

In [68]:
# Compute Laplacian matrix of mst3
laplacian_matrix = degree_matrix - adj_matrix

# Print Laplacian matrix
print(laplacian_matrix.shape)
print(laplacian_matrix)

(72, 72)
[[ 8.  0.  0. ...  0.  0. -1.]
 [ 0.  4.  0. ...  0.  0.  0.]
 [ 0.  0.  5. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 12.  0.  0.]
 [ 0.  0.  0. ...  0.  7. -1.]
 [-1.  0.  0. ...  0. -1. 12.]]


*Eigenvectors and eigenvalues*

In [69]:
# Compute first k eigen vectors of Laplacian matrix where k is number of clusters
k = 2
eigen_values, eigen_vectors = np.linalg.eig(laplacian_matrix)
eigen_vectors = eigen_vectors[:, eigen_values.argsort()]
U = eigen_vectors[:, :k]

# Print eigen vectors
print(U.shape)
print(U)

(72, 2)
[[ 0.118  0.01 ]
 [ 0.118  0.06 ]
 [ 0.118  0.304]
 [ 0.118  0.018]
 [ 0.118  0.064]
 [ 0.118  0.349]
 [ 0.118 -0.065]
 [ 0.118  0.012]
 [ 0.118  0.343]
 [ 0.118  0.272]
 [ 0.118  0.254]
 [ 0.118 -0.051]
 [ 0.118  0.037]
 [ 0.118  0.126]
 [ 0.118  0.067]
 [ 0.118  0.071]
 [ 0.118  0.116]
 [ 0.118  0.01 ]
 [ 0.118  0.012]
 [ 0.118  0.091]
 [ 0.118  0.059]
 [ 0.118  0.012]
 [ 0.118  0.362]
 [ 0.118  0.06 ]
 [ 0.118 -0.041]
 [ 0.118  0.011]
 [ 0.118 -0.011]
 [ 0.118 -0.018]
 [ 0.118 -0.09 ]
 [ 0.118 -0.05 ]
 [ 0.118 -0.054]
 [ 0.118 -0.018]
 [ 0.118 -0.036]
 [ 0.118 -0.027]
 [ 0.118 -0.132]
 [ 0.118 -0.057]
 [ 0.118 -0.045]
 [ 0.118 -0.061]
 [ 0.118 -0.031]
 [ 0.118 -0.022]
 [ 0.118 -0.052]
 [ 0.118  0.016]
 [ 0.118 -0.139]
 [ 0.118 -0.079]
 [ 0.118 -0.062]
 [ 0.118 -0.081]
 [ 0.118 -0.151]
 [ 0.118  0.075]
 [ 0.118  0.002]
 [ 0.118 -0.042]
 [ 0.118 -0.099]
 [ 0.118 -0.014]
 [ 0.118 -0.049]
 [ 0.118 -0.112]
 [ 0.118 -0.135]
 [ 0.118 -0.149]
 [ 0.118 -0.123]
 [ 0.118 -0.153]
 [ 0.1

*K-means Clustering*

In [70]:
def doKmeans(X, nclust):
    model = KMeans(nclust)
    model.fit(X)
    clust_labels = model.predict(X)
    cent = model.cluster_centers_
    return (clust_labels, cent)

start = time.time()
clust_labels, cent = doKmeans(U, 2)
end = time.time()
score=silhouette_score(U, clust_labels,metric='euclidean')
print("Silhouette Score: ", score)
print("Execution time: ", end-start)

Silhouette Score:  0.7467399520680679
Execution time:  0.028738021850585938


*Computing Complete Graph of U*

In [71]:
# Compute Complete Graph of U
G = nx.Graph()
dist_matrix = squareform(pdist(U))

# Print adjacency matrix
print(dist_matrix.shape)
print(dist_matrix)


(72, 72)
[[0.    0.051 0.295 ... 0.107 0.008 0.008]
 [0.051 0.    0.244 ... 0.157 0.059 0.058]
 [0.295 0.244 0.    ... 0.401 0.303 0.303]
 ...
 [0.107 0.157 0.401 ... 0.    0.098 0.099]
 [0.008 0.059 0.303 ... 0.098 0.    0.001]
 [0.008 0.058 0.303 ... 0.099 0.001 0.   ]]


*Computing MST of U*

In [72]:
# Compute MST of U
mst = MST(dist_matrix)

# Add weights to the edges of the MST
for i in range(len(mst)):
    mst[i].append(dist_matrix[mst[i][0]][mst[i][1]])

# Sort the edges
mst.sort(key=lambda x: x[2])

*MST-based Clustering*

In [73]:
sorted_edges = mst
n = len(nodes)
e = len(sorted_edges)
visited = [0 for i in range(n)]
adjMatrix =[[-1 for i in range(n)] for i in range(n)]

def traverse(start,adjMatrix,visited,c):
    visited[start] = c
    for i in range(n):
        if(visited[i]==0 and adjMatrix[start][i]!=-1):
            traverse(i,adjMatrix,visited,c)

# removing k longest edges
start = time.time()
k = 2
clusters=[]
for i in range(e-k):
    adjMatrix[sorted_edges[i][0]][sorted_edges[i][1]] = sorted_edges[i][2]
    adjMatrix[sorted_edges[i][1]][sorted_edges[i][0]] = sorted_edges[i][2]
c = 0
for i in range(n):
    if visited[i] == 0:
        c += 1
        traverse(i,adjMatrix,visited,c)
e=len(sorted_edges)
end=time.time()
clust_labels = visited
score=silhouette_score(U, clust_labels, metric='euclidean')
print("Silhouette Score: ", score)
print("Execution time:",end-start)

Silhouette Score:  0.6973274621762604
Execution time: 0.001252889633178711
